<a href="https://colab.research.google.com/github/ridaCham/mnist_intern/blob/main/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.datasets import mnist
from keras.applications.mobilenet import MobileNet
from keras.models import Sequential
from keras.layers import Dense, Flatten


In [9]:
# Veri kümesini yükle
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Görüntüleri 0-1 arasında ölçeklendirin ve 3 kanallı hale getirin
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
train_images = np.stack((train_images,) * 3, axis=-1)
test_images = np.stack((test_images,) * 3, axis=-1)



train_images = np.array([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in train_images])
test_images = np.array([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in test_images])


In [10]:
conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

model = Sequential([
    conv_base,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 64)                65600     
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 3,295,114
Trainable params: 3,273,226
Non-trainable params: 21,888
_________________________________________________________________


In [11]:
# Modeli eğitin
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))


Epoch 1/5
938/938 [==============================] - 726s 761ms/step - loss: 2.0021 - accuracy: 0.2788 - val_loss: 1.9318 - val_accuracy: 0.3021
Epoch 2/5
938/938 [==============================] - 710s 758ms/step - loss: 1.7810 - accuracy: 0.3655 - val_loss: 1.8143 - val_accuracy: 0.3452
Epoch 3/5
938/938 [==============================] - 709s 756ms/step - loss: 1.7306 - accuracy: 0.3815 - val_loss: 1.7153 - val_accuracy: 0.3901
Epoch 4/5
938/938 [==============================] - 707s 754ms/step - loss: 1.7018 - accuracy: 0.3919 - val_loss: 1.7031 - val_accuracy: 0.3912
Epoch 5/5
938/938 [==============================] - 713s 761ms/step - loss: 1.6848 - accuracy: 0.3975 - val_loss: 1.6768 - val_accuracy: 0.3987


In [12]:

score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


model.save('mobilenet.h5')

Test loss: 1.6768027544021606
Test accuracy: 0.3986999988555908
